In [ ]:
pip install tqdm

In [ ]:
!pip install tiktoken


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.2 MB/s eta 0:00:00


In [ ]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00


In [ ]:
import json
import pandas as pd

In [ ]:
# DELETE

fold_num = "fold_5"
# Load the data from each fold
X_train = pd.read_csv(f'{fold_num}_train.csv')
y_train = pd.read_csv(f'{fold_num}_train_labels.csv')
X_test = pd.read_csv(f'{fold_num}_test.csv')
y_test = pd.read_csv(f'{fold_num}_test_labels.csv')


X_train['text'] = X_train
X_test['text'] = X_test

X_train['labels'] = y_train
X_test['labels'] = y_test

X_train = X_train.drop(['def'], axis=1)
X_test = X_test.drop(['def'], axis=1)

In [8]:

# Set the fold number
fold_num = "fold_5"

# Load the data from each fold
X_test = pd.read_csv(f'{fold_num}_test.csv')
y_test = pd.read_csv(f'{fold_num}_test_labels.csv')


X_test['text'] = X_test
X_test['labels'] = y_test
X_test = X_test.drop(['def'], axis=1)

In [9]:
# Including the direct relations dataset to be
# able to have a sample with as many indirect and direct instances we want
direct = pd.read_csv('direct_relations.csv')
direct["text"] = direct["Definição_Synset"] + "[SEP]" + direct["Definição_Relacionada"]

In [10]:
dataset_reduced = X_test

In [11]:


# Add a 'type' column to X_train and X_test, initialized with 'indirect'
dataset_reduced['type'] = 'indirect'
dataset_reduced['type'] = 'indirect'

# Set 'type' to 'direct' for rows in X_train and X_test that are present in direct
dataset_reduced.loc[dataset_reduced['text'].isin(direct['text']), 'type'] = 'direct'
dataset_reduced.loc[dataset_reduced['text'].isin(direct['text']), 'type'] = 'direct'


In [13]:
dataset_reduced['label+type'] = dataset_reduced['labels'].astype(str) + dataset_reduced['type']

In [14]:

from sklearn.utils import resample
instances_per_class = 400
# Assuming 'labels' is the column with your labels
unique_labels = dataset_reduced['label+type'].unique()
sample_size_per_label = (instances_per_class * 5) // len(unique_labels)

sampled_data = pd.DataFrame()
for label in unique_labels:
  label_data = dataset_reduced[dataset_reduced['label+type'] == label]
  sampled_label_data = resample(label_data, replace=False, n_samples=sample_size_per_label, random_state=42)
  sampled_data = pd.concat([sampled_data, sampled_label_data])

# Shuffle the sampled data
sampled_data = sampled_data.sample(frac=1, random_state=42).reset_index(drop=True)

print(sampled_data)


                                                   text     labels      type  \
0     (Roman mythology) the twin brother of Romulus[...  Unrelated  indirect   
1     a depression hollowed out of solid matter[SEP]...  Hypernyms    direct   
2     any tree of the division Gymnospermophyta[SEP]...   Holonyms  indirect   
3     pale yellow seedless grape used for raisins an...  Hypernyms  indirect   
4     a tall perennial woody plant having a main tru...   Holonyms  indirect   
...                                                 ...        ...       ...   
1995  any period of time during which glaciers cover...  Hypernyms  indirect   
1996  an entity that has physical existence[SEP]mult...   Holonyms  indirect   
1997  a generic name for digital lines that are prov...  Hypernyms  indirect   
1998  a living thing that has (or can develop) the a...   Holonyms  indirect   
1999  a government order imposing a trade barrier[SE...  Hypernyms  indirect   

             label+type  
0     Unrelat

In [15]:
# Cheack the label distribution for each class

sampled_data['label+type'].value_counts()


,count
label+type,
Unrelatedindirect,400
Hypernymsdirect,400
Holonymsindirect,400
Hypernymsindirect,400
Holonymsdirect,400


In [18]:
# If we want to have the same amount fro unrelated and hypernym direct + indirect and holonym direct + indirect, perform this step

# Filter the DataFrame for the specified labels
hypernyms_direct = sampled_data[sampled_data['label+type'] == 'Hypernymsdirect']
holonyms_indirect = sampled_data[sampled_data['label+type'] == 'Holonymsindirect']
hypernyms_indirect = sampled_data[sampled_data['label+type'] == 'Hypernymsindirect']
holonyms_direct = sampled_data[sampled_data['label+type'] == 'Holonymsdirect']

# Randomly sample half of the rows from each filtered DataFrame
hypernyms_direct_sampled = hypernyms_direct.sample(frac=0.5, random_state=42)
holonyms_indirect_sampled = holonyms_indirect.sample(frac=0.5, random_state=42)
hypernyms_indirect_sampled = hypernyms_indirect.sample(frac=0.5, random_state=42)
holonyms_direct_sampled = holonyms_direct.sample(frac=0.5, random_state=42)

# Get the indices of the rows to remove
indices_to_remove = hypernyms_direct_sampled.index.tolist() + holonyms_indirect_sampled.index.tolist() + hypernyms_indirect_sampled.index.tolist() + holonyms_direct_sampled.index.tolist()

# Remove the rows from the original DataFrame
sampled_data_reduced = sampled_data.drop(indices_to_remove)

print(sampled_data_reduced)


                                                   text     labels      type  \
0     (Roman mythology) the twin brother of Romulus[...  Unrelated  indirect   
4     a tall perennial woody plant having a main tru...   Holonyms  indirect   
5     (chemistry) a substance formed by chemical uni...   Holonyms  indirect   
6     a trumpet-shaped acoustic device to intensify ...  Hypernyms  indirect   
7     a layer of cells on the inside of the blastula...  Unrelated  indirect   
...                                                 ...        ...       ...   
1991  small Old World finch whose male has a red bre...  Unrelated  indirect   
1993  a fortified hilly area between southern Lebano...  Unrelated  indirect   
1995  any period of time during which glaciers cover...  Hypernyms  indirect   
1998  a living thing that has (or can develop) the a...   Holonyms  indirect   
1999  a government order imposing a trade barrier[SE...  Hypernyms  indirect   

             label+type  
0     Unrelat

In [ ]:
# Save the sampled data to a csv file
sampled_data_reduced.to_csv('sampled_data_reduced.csv', index=False)

In [ ]:
sampled_data.to_csv('sampled_data.csv', index=False)

In [19]:
dataset_reduced = sampled_data_reduced

In [20]:
# Partition 'text' column on the first occurrence of 'SEP'
dataset_reduced[['def1', 'sep', 'def2']] = dataset_reduced['text'].str.partition('[SEP]')

# Drop the 'sep' column as it is just the separator
dataset_reduced = dataset_reduced.drop(columns=['sep'])

# Optionally, remove the first 4 characters from 'def2'
dataset_reduced['def2'] = dataset_reduced['def2']


You are a language expert and must assist in a task of defining the type of relationship between two concepts, considering their definitions in natural language.

Consider two concepts c1 and c2. We say that the relationship between them is hypernym when every instance of c2 is also an instance of c1, because c1 generalizes c2. We say that there is a holonym relationship when instances of c2 are parts of instances of c1. And we say that they are not related when there is neither a hypernym nor a holonym relationship between them. Next, I will provide definitions for the concepts C1 and C2 and you should respond with just a number that identifies the relationship between them: 0 indicates a holonym relationship, 1 indicates a hypernym relationship, and 3 indicates that neither of the two relationships exists between them:

## Batch approach

In [46]:
import os
import json
import shutil
from google.colab import files

try:
    import tiktoken
except ImportError:
    import os
    os.system('pip install tiktoken')
    import tiktoken

# Initialize the tokenizer with a known encoding
tokenizer = tiktoken.get_encoding("cl100k_base")

def count_tokens(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

tasks = []
df = dataset_reduced

# Define the system prompt outside the loop
categorize_system_prompt = "You are a helpful assistant, specialized in linguistics, ontologies and knowledge graphs and you must help classify the type of relationship that exists between a certain pair of concepts, considering their definitions."

# Count tokens for the system prompt
system_prompt_tokens = count_tokens(categorize_system_prompt)

# Initialize token counters and batch count
current_batch_tokens = 0
batch_count = 0
total_batches = 0
batch_files = []

total_prompt_tokens = 0
total_completion_tokens = 0
batch_size_token_limit = 400 #choose according to your account limits

# Create the batches folder if it doesn't exist
batch_folder = "batches"
if not os.path.exists(batch_folder):
    os.makedirs(batch_folder)

for index, row in df.iterrows():
    # Retrieve definitions for each row
    def1 = row['def1']
    def2 = row['def2']

    # Prepare description for user interaction
    description = f"""
    Consider two concepts c1 and c2. We say that the relationship between
them is hyperonymy when an instance of c1 is also an instance of c2, because
c2 generalizes c1. We say that there is a holonym relationship when instances of
c1 are parts of instances of c2. And we say that they are not related when there
is neither a hyperonymy nor a holonym relationship between them.
Next, I will provide definitions for the concepts C1 and C2 and you should respond with just a number that identifies the relationship between them: 0 indicates a holonym relationship, 1 indicates a hyperonymy relationship, and 3 indicates that neither of the two relationships exists between them

The definition of c1:{def1} and the definition of c2:{def2}"""

    # Count tokens for the user prompt
    user_prompt_tokens = count_tokens(description)

    # Assuming a fixed completion length of 10 tokens for this example (adjust as needed)
    completion_length = 1

    # Total tokens for this request
    total_request_tokens = system_prompt_tokens + user_prompt_tokens + completion_length

    # Check if adding this task would exceed the batch token limit
    if current_batch_tokens + total_request_tokens > batch_size_token_limit:
        # Write current batch to file
        file_name = f"{batch_folder}/batch_{batch_count + 1}_examples.jsonl"
        with open(file_name, 'w') as file:
            for obj in tasks:
                file.write(json.dumps(obj) + '\n')
        # Append the filename to the batch files list
        batch_files.append(file_name)
        # Clear the tasks list and increment batch count
        tasks = []
        current_batch_tokens = 0
        batch_count += 1
        total_batches += 1

    # Add task to current batch
    task = {
        "custom_id": f"task-{index}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            # Chat Completions API call parameters
            "model": "gpt-4o",
            "temperature": 0.1,
            "response_format": {
                "type": "text"
            },
            "messages": [
                {
                    "role": "system",
                    "content": categorize_system_prompt
                },
                {
                    "role": "user",
                    "content": description
                }
            ],
        }
    }
    tasks.append(task)
    current_batch_tokens += total_request_tokens

    # Update the total prompt and completion tokens
    total_prompt_tokens += (system_prompt_tokens + user_prompt_tokens)
    total_completion_tokens += completion_length

    # Print the token counts for each task
    print(f"Task {index}: System Prompt Tokens = {system_prompt_tokens}, User Prompt Tokens = {user_prompt_tokens}, Total Tokens = {total_request_tokens}")

# Check if there are any remaining tasks
if tasks:
    file_name = f"{batch_folder}/batch_{batch_count + 1}_examples.jsonl"
    with open(file_name, 'w') as file:
        for obj in tasks:
            file.write(json.dumps(obj) + '\n')
    batch_files.append(file_name)
    total_batches += 1

# Calculate costs
cost_per_1M_input_tokens = 5.00
cost_per_1M_output_tokens = 15.00

total_input_tokens = total_prompt_tokens  # System + User Prompts
total_output_tokens = total_completion_tokens  # Completions

cost_of_inputs = (total_input_tokens / 1_000_000) * cost_per_1M_input_tokens
cost_of_outputs = (total_output_tokens / 1_000_000) * cost_per_1M_output_tokens
total_cost = cost_of_inputs + cost_of_outputs

print(f"Total Input Tokens: {total_input_tokens}")
print(f"Total Output Tokens: {total_output_tokens}")
print(f"Cost of Inputs: ${cost_of_inputs:.2f}")
print(f"Cost of Outputs: ${cost_of_outputs:.2f}")
print(f"Total Cost: ${total_cost:.2f}")

# Print total number of batches
print(f"Total Number of Batches: {total_batches}")

# Print the names of batch files for download
print("Batch files created:")
for file_name in batch_files:
    print(file_name)

# Create a ZIP archive of the batch files
shutil.make_archive('/content/batches_archive', 'zip', batch_folder)

# Provide a download link
files.download('/content/batches_archive.zip')


Task 0: System Prompt Tokens = 38, User Prompt Tokens = 198, Total Tokens = 237
Task 4: System Prompt Tokens = 38, User Prompt Tokens = 218, Total Tokens = 257
Task 5: System Prompt Tokens = 38, User Prompt Tokens = 200, Total Tokens = 239
Task 6: System Prompt Tokens = 38, User Prompt Tokens = 198, Total Tokens = 237
Task 7: System Prompt Tokens = 38, User Prompt Tokens = 182, Total Tokens = 221
Task 8: System Prompt Tokens = 38, User Prompt Tokens = 202, Total Tokens = 241
Task 10: System Prompt Tokens = 38, User Prompt Tokens = 186, Total Tokens = 225
Task 11: System Prompt Tokens = 38, User Prompt Tokens = 201, Total Tokens = 240
Task 12: System Prompt Tokens = 38, User Prompt Tokens = 195, Total Tokens = 234
Task 15: System Prompt Tokens = 38, User Prompt Tokens = 181, Total Tokens = 220
Task 16: System Prompt Tokens = 38, User Prompt Tokens = 216, Total Tokens = 255
Task 18: System Prompt Tokens = 38, User Prompt Tokens = 199, Total Tokens = 238
Task 19: System Prompt Tokens = 38

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [47]:
from openai import OpenAI

client = openai.OpenAI(api_key= 'your-api-key-here')

batch_input_file = client.files.create(
  file=open("/content/batches/batch_1_examples.jsonl", "rb"),
  purpose="batch"
)

In [48]:
batch_input_file_id = batch_input_file.id

client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "batch"
    }
)

Batch(id='batch_tVdqkZS19s3N7kAh7Klh4qtY', completion_window='24h', created_at=1724230643, endpoint='/v1/chat/completions', input_file_id='file-lSxdlhcJ5CNFY4eXTFo4vqqR', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1724317043, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'batch'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [66]:

# List all batches to find the latest
batches = client.batches.list()

# Assuming you want the most recently created batch
latest_batch = batches.data[0]
batch_id = latest_batch.id

print(batch_id)


batch_tVdqkZS19s3N7kAh7Klh4qtY


In [67]:
client.batches.retrieve(batch_id)

Batch(id='batch_tVdqkZS19s3N7kAh7Klh4qtY', completion_window='24h', created_at=1724230643, endpoint='/v1/chat/completions', input_file_id='file-lSxdlhcJ5CNFY4eXTFo4vqqR', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1724230706, error_file_id=None, errors=None, expired_at=None, expires_at=1724317043, failed_at=None, finalizing_at=1724230705, in_progress_at=1724230644, metadata={'description': 'batch'}, output_file_id='file-KVvZrthBfJzwjA70MFCaOgpm', request_counts=BatchRequestCounts(completed=1, failed=0, total=1))

In [71]:

batch = client.batches.retrieve(batch_id)
if batch.status == "completed":
  print("Batch is completed.")
  batch_output_file_id = batch.output_file_id
else:
  print("Batch is not yet completed.")



Batch is completed.


In [69]:
file_response = client.files.content(batch_output_file_id)
print(file_response.text)

{"id": "batch_req_lcLO79oUNSL17fqwypyqC2h2", "custom_id": "task-0", "response": {"status_code": 200, "request_id": "1fd162e9c46ed10cb05cbbf03beeb8c2", "body": {"id": "chatcmpl-9ybSb1mQx0Ef9hidKc2zwSE8JQcUR", "object": "chat.completion", "created": 1724230705, "model": "gpt-4o-2024-05-13", "choices": [{"index": 0, "message": {"role": "assistant", "content": "3", "refusal": null}, "logprobs": null, "finish_reason": "stop"}], "usage": {"prompt_tokens": 248, "completion_tokens": 1, "total_tokens": 249}, "system_fingerprint": "fp_3aa7262c27"}}, "error": null}



### Using open ai interface

If you are using the open ai interface, upload the batch files to https://platform.openai.com/batches

In [32]:
import json

file_name =  'your_batch_file_name'

file_path = f'/content/{file_name}'

# Read the JSONL file line by line
responses = []
with open(file_path, 'r') as file:
    for line in file:
        responses.append(json.loads(line))


# Extract the numbers from the content fields
numbers = [int(item['response']['body']['choices'][0]['message']['content']) for item in responses]

print(numbers)
# Output the responses to understand their structure


[3, 3, 3, 3, 1, 3, 3, 3, 1, 3, 3, 1, 1, 3, 3, 1, 3, 1, 3, 3, 1, 1, 1, 1, 3, 1, 3, 3, 3, 1, 3, 3, 3, 3, 1, 0, 1, 1, 3, 1, 3, 3, 3, 3, 3, 3, 3, 1, 3, 1, 3, 3, 1, 3, 3, 1, 3, 3, 3, 1, 1, 3, 1, 1, 1, 3, 3, 3, 3, 3, 0, 3, 3, 1, 3, 3, 1, 1, 3, 3, 3, 3, 3, 1, 1, 3, 3, 3, 3, 3, 1, 0, 1, 1, 3, 0, 3, 3, 1, 1, 1, 3, 1, 1, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 3, 3, 3, 1, 3, 1, 1, 3, 3, 1, 3, 1, 1, 3, 3, 1, 3, 3, 3, 3, 3, 1, 3, 0, 1, 1, 1, 3, 1, 3, 3, 1, 1, 3, 3, 0, 1, 3, 3, 3, 3, 3, 1, 1, 3, 3, 3, 3, 1, 1, 3, 1, 1, 1, 1, 1, 3, 1, 1, 3, 3, 1, 3, 1, 3, 1, 3, 3, 1, 3, 1, 1, 3, 3, 1, 3, 3, 3, 1, 3, 3, 3, 3, 3, 1, 0, 1, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 3, 1, 3, 3, 3, 1, 3, 1, 3, 1, 3, 3, 3, 3, 1, 3, 1, 3, 1, 3, 1, 3, 3, 3, 1, 1, 3, 1, 3, 3, 1, 3, 1, 3, 1, 1, 3, 3, 3, 3, 3, 1, 3, 1, 3, 1, 1, 3, 1, 1, 1, 1, 3, 3, 3, 3, 1, 3, 3, 3, 1, 3, 3, 3, 1, 3, 3, 3, 3, 0, 0, 3, 3, 3]


### Evaluate

In [37]:

from sklearn.metrics import classification_report


# Convert result_df to numerical labels if they are not already
result_df_numeric =  numbers

# Calculate and print the classification report
report = classification_report(dataset_reduced['labels'], result_df_numeric, zero_division=1)
print(report)


              precision    recall  f1-score   support

           0       0.44      0.04      0.07        99
           1       0.32      0.38      0.35        91
           3       0.38      0.64      0.48       110

    accuracy                           0.36       300
   macro avg       0.38      0.35      0.30       300
weighted avg       0.39      0.36      0.31       300



## Request approach


In [ ]:
import openai
from tqdm import tqdm

openai.api_key = 'your-api-key-here'

# Create a client instance
client = openai
data = dataset_reduced
result_df = []
df = data

# Define the system prompt outside the loop
categorize_system_prompt = f"""
You are a helpful assistant, specialized in linguistics, ontologies and knowledge graphs and you must help classify the type of relationship that exists between a certain pair of concepts, considering their definitions.

"""

# Initialize the tqdm progress bar with the number of rows in the dataframe
for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing"):
    # Retrieve definitions for each row
    def1 = row['def1']
    def2 = row['def2']

    # Prepare description for user interaction
    description = f"""
    Consider two concepts c1 and c2. We say that the relationship between
them is hyperonymy when an instance of c1 is also an instance of c2, because
c2 generalizes c1. We say that there is a holonym relationship when instances of
c1 are parts of instances of c2. And we say that they are not related when there
is neither a hyperonymy nor a holonym relationship between them.
Next, I will provide definitions for the concepts C1 and C2 and you should respond with just a number that identifies the relationship between them: 0 indicates a holonym relationship, 1 indicates a hyperonymy relationship, and 3 indicates that neither of the two relationships exists between them

The definition of c1:{def1} and the definition of c2:{def2}"""


    # Make the API call for each task
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": categorize_system_prompt
            },
            {
                "role": "user",
                "content": description
            }
        ],
        temperature=0,
        max_tokens=1
    )

    # Extract and process the response as needed
    result = response.choices[0].message.content
    result_df.append(result)


Processing: 100%|██████████| 1200/1200 [08:47<00:00,  2.27it/s]


In [34]:

label_mapping = {"Hypernyms": 1, "Holonyms": 0, "Unrelated": 3}  # Define your label mapping

# Assuming 'labels' is the column in X_test containing the labels
dataset_reduced['labels'] = dataset_reduced['labels'].map(label_mapping)

print(dataset_reduced.head())


                                                text  labels      type  \
0  (Roman mythology) the twin brother of Romulus[...       3  indirect   
4  a tall perennial woody plant having a main tru...       0  indirect   
5  (chemistry) a substance formed by chemical uni...       0  indirect   
6  a trumpet-shaped acoustic device to intensify ...       1  indirect   
7  a layer of cells on the inside of the blastula...       3  indirect   

          label+type                                               def1  \
0  Unrelatedindirect      (Roman mythology) the twin brother of Romulus   
4   Holonymsindirect  a tall perennial woody plant having a main tru...   
5   Holonymsindirect  (chemistry) a substance formed by chemical uni...   
6  Hypernymsindirect  a trumpet-shaped acoustic device to intensify ...   
7  Unrelatedindirect     a layer of cells on the inside of the blastula   

                                                def2  
0  a morpheme that occurs only as part of a large

In [ ]:

from sklearn.metrics import classification_report


# Convert result_df to numerical labels if they are not already
result_df_numeric = [int(x) if x.isdigit() else None for x in result_df]

# Calculate and print the classification report
report = classification_report(dataset_reduced['labels'], result_df_numeric, zero_division=1)
print(report)


              precision    recall  f1-score   support

           0       0.81      0.09      0.15       400
           1       0.63      0.54      0.58       400
           3       0.49      1.00      0.66       400

    accuracy                           0.54      1200
   macro avg       0.64      0.54      0.46      1200
weighted avg       0.64      0.54      0.46      1200

